In [ ]:
for i in range(20):
    ser.write(b'hello world')

In [ ]:
ser.write(b'hello world')

# Setup

In [13]:
import time
import serial
ser = serial.Serial(port='/dev/serial0',baudrate=9600,parity=serial.PARITY_NONE,stopbits=serial.STOPBITS_ONE,bytesize=serial.EIGHTBITS,timeout=1)

In [8]:
from enum import Enum

VERBOSITY_QUIET = 0
VERBOSITY_DEBUG = 1
Global_Verbosity = VERBOSITY_QUIET


def set_Verbosity(verbosity):
    Global_Verbosity = verbosity


def printLog(str, verbosity):
    if verbosity <= Global_Verbosity:
        print(str)

class eMsgType(Enum):
    MSG_MODE_CONTROL = b'\x10'
    MSG_COORDINATE = b'\x20'
    MSG_ACK = b'\x40'

class eSepChar(Enum): #start and ending hexes, acknowledgement bit
    STX_CHAR = b'\x02'
    ETX_CHAR = b'\x03'
    ACK_ASCII = b'ACK' 
    UNUSED = b'\xFF'

class eFlagNumber(Enum): #to compare what we're reading
    OFF = 0
    AUTO_AIM_MODE = 1 
    AUTO_MOVE_MODE = 2 
    ENEMY_MODE=4

class eReaderState(Enum):
    READ_STX = 0
    READ_PAYLOAD = 1
    READ_ERROR = 2

def send_coordinates(x,y,xspeed,yspeed): #need to update this
	CoordinateData=eSepChar.STX_CHAR.value + eMsgType.MSG_COORDINATE.value + x.to_bytes(2,'little')+y.to_bytes(2,'little')+xspeed.to_bytes(32,'little')+yspeed.to_bytes(32,'little')+eSepChar.ETX_CHAR.value
	print("Printing coordinate data")
	print(CoordinateData)
	ser.write(CoordinateData)


In [7]:
def AutoAim_Heartbeat_OLDFORREFFERENCE(x,y,xspeed,yspeed,verbosity):
    #this is cv, arduino is control
	try: #try to print your current state - if you can, they must be defined, so print them
		printLog(AutoAim_Heartbeat.Heartbeat_State,VERBOSITY_DEBUG)
		printLog(AutoAim_Heartbeat.Reader_State,VERBOSITY_DEBUG)
	except: #if not, they are undefined, so go ahead and define them
		AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_INIT
		AutoAim_Heartbeat.Reader_State = eReaderState.READ_STX
	
	if AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_INIT:
		ser.reset_input_buffer()
		ser.reset_output_buffer()
		print("Reactor online. Sensors online. Weapons online. All systems nominal.\n")
		send_coordinates(x,y,xspeed,yspeed)
		AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_IDLE
		msgTime=time.time()
	elif AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_IDLE:     
		if ser.inWaiting()>0:
			msgTime=time.time();
			if AutoAim_Heartbeat.Reader_State == eReaderState.READ_STX: 
				if ser.read(1) == eSepChar.STX_CHAR.value: #check if the first bit is the STX value
					AutoAim_Heartbeat.Reader_State = eReaderState.READ_PAYLOAD	#if yes, enter reading state
			if AutoAim_Heartbeat.Reader_State == eReaderState.READ_PAYLOAD: 
				msgType = ser.read(1)
				if msgType == eMsgType.MODE_CONTROL.value: #check if the bit you are recieving is telling you to change mode
					newState = int(ser.read(1)) #read the next bit
					ACKData = eSepChar.STX_CHAR.value + eSepChar.ACK.value + eSepChar.ACK_ASCII.value + eSepChar.UNUSED.value + eSepChar.UNUSED.value + eSepChar.UNUSED.value +eSepChar.UNUSED.value +eSepChar.UNUSED.value +eSepChar.ETX_CHAR.value
					if newState==eAutoAimState.AUTO_AIM_INIT: 
						AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_INIT
						print("I am changing my mode to initialization\n") 
						print(ACKData)
						ser.write(ACKData)
					elif newState==eAutoAimState.AUTO_AIM_IDLE:                  
						AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_IDLE
						print("I am changing my mode to idle\n") 
						print(ACKData)
						ser.write(ACKData)
                    elif newState==eAutoAimState.AUTO_AIM_ENEMY_MODE:                  
						AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_ENEMY_MODE
						print("I am changing my mode to enemy mode.\n") 
						print(ACKData)
                    elif newState==eAutoAimState.AUTO_AIM_BOTH:                  
						AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_BOTH
						print("I am changing my mode to idle\n") 
						print(ACKData)    
					elif newState==eAutoAimState.AUTO_AIM_SLEEP:
						AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_SLEEP
						print("I am changing my mode to waiting for sleep\n") 
						print(ACKData)
						ser.write(ACKData)
				if msgType == eMsgType.COORDINATE:
                    
                else:
					AutoAim_Heartbeat.Reader_State = eReaderState.READ_ERROR #read something wrong, throw error
			else:
				printLog("Unexpected AutoAim API state!")
				raise RuntimeError
		else:
			if(time.time() - msgTime > 5):
			print("timed out.")   
			AutoAim_Heartbeat.Heartbeat_State = eAutoAimState.AUTO_AIM_SLEEP;            
	elif AutoAim_Heartbeat.Heartbeat_State == eAutoAimState.AUTO_AIM_SLEEP:
	    #send acknowledgement
		ser.write(eSepChar.STX_CHAR + eMsgType.MODE_CONTROL + eSepChar.ACK_ASCII + eSepChar.ETX_CHAR ) 
		print("Auto aim disabled.")
	else:
		printLog("Unexpected AutoAim API state!")
		raise RuntimeError


	# CV should turn off if state is not active to save battery
	fCvActive = not (AutoAim_Heartbeat.Heartbeat_State in (eAutoAimState.AUTO_AIM_IDLE,eAutoAimState.AUTO_AIM_SLEEP))
	return fCvActive

#for the sake of testing 
AutoAim_Heartbeat(69,42,69,42,1)

IndentationError: unindent does not match any outer indentation level (<ipython-input-7-d3ae6de6eef1>, line 38)

In [ ]:
bytes(0x02).decode()

In [14]:
#flags. IDK how global variables work with jupyter's cellular structure but this doesnt crash so
autoAim=False
autoMove=False
EnemyMode=False

def AutoAim_Heartbeat(x,y,xspeed,yspeed,verbosity):
    #this is cv, arduino is control
	global autoAim
	global autoMove
	global EnemyMode    
	try: #try to print your current state - if you can, they must be defined, so print them
		printLog(AutoAim_Heartbeat.Reader_State,VERBOSITY_DEBUG)
	except: #if not, they are undefined, so go ahead and define them
		AutoAim_Heartbeat.Reader_State = eReaderState.READ_STX
	if (autoAim):
		ser.reset_input_buffer()
		ser.reset_output_buffer()
		send_coordinates(x,y,xspeed,yspeed) #send x y coordinates
	if autoMove:
		ser.reset_input_buffer()
		ser.reset_output_buffer()
		send_coordinates(x,y,xspeed,yspeed) #send x y speed
	if EnemyMode:
		ser.reset_input_buffer()
		ser.reset_output_buffer()
		#placeholder  
	if ser.inWaiting()>0:
		if AutoAim_Heartbeat.Reader_State == eReaderState.READ_STX: 
			if ser.read(1) == eSepChar.STX_CHAR.value: #check if the first bit is the STX value
				AutoAim_Heartbeat.Reader_State = eReaderState.READ_PAYLOAD	#if yes, enter reading state
		if AutoAim_Heartbeat.Reader_State == eReaderState.READ_PAYLOAD: 
			msgType = int(ser.read(1))
			if msgType == eMsgType.MODE_CONTROL.value: #check if the bit you are recieving is telling you to change mode
				newState = int(ser.read(1)) #read the next bit
				ACKData = eSepChar.STX_CHAR.value + eSepChar.ACK_ASCII.value + eSepChar.UNUSED.value + eSepChar.UNUSED.value + eSepChar.UNUSED.value +eSepChar.UNUSED.value +eSepChar.UNUSED.value +eSepChar.ETX_CHAR.value
				if newState==eAutoAimState.AUTO_AIM_ON: 
					autoAim= !autoAim;
					if autoAim:
						autoMove=False
						EnemyMode=False
					print("Toggling on auto aim\n") 
					print("AutoAim: "+autoAim)                    
					print(ACKData)
					ser.write(ACKData)
				elif newState==eAutoAimState.AUTO_MOVE_MODE:  
					autoMove= !autoMove
					if autoMove:
						autoAim=False #mutually exclusive
						EnemyMode=False
					print("Toggling on auto move\n") 
					print("AutoMove: "+autoMove)                    
					print(ACKData)
					ser.write(ACKData)
				if newState==eEnemyMode_State.ENEMY_MODE_ON:
					EnemyMode=!EnemyMode;                    
					if EnemyMode:
						autoAim=False #mutually exclusive
						EnemyMode=False                    
					print("Toggling on auto move\n") 
					print("AutoMove: "+autoMove) 
					print(ACKData)
					ser.write(ACKData)
			else:
				AutoAim_Heartbeat.Reader_State = eReaderState.READ_ERROR #read something wrong, throw error
		else:
			printLog("Unexpected AutoAim API state!")
			raise RuntimeError
	return  (autoAim,autoMove,EnemyMode)       

AutoAim_Heartbeat(69,42,69,42,1)


(False, False, False)

## Unit test

In [ ]:
#AutoAim_Heartbeat(10,10,1)
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))